In [ ]:
import socket
import struct
import threading
import cv2
import numpy as np
import time
import sys

# === 패킷 클래스 정의 ===
class Packet:
    HEADER_FORMAT = '2s'  # Command ID (2바이트), 엔디언 지정 없음
    HEADER_SIZE = struct.calcsize(HEADER_FORMAT)

    def __init__(self, command_id, data=b''):
        self.command_id = command_id  # 'MO', 'DE', 'SF'
        self.data = data  # 가변 길이의 바이트 데이터

    def to_bytes(self):
        # 패킷을 bytes로 변환: command_id + data + \n
        header = struct.pack(self.HEADER_FORMAT, self.command_id.encode())
        return header + self.data + b'\n'

    @classmethod
    def from_bytes(cls, data):
        if len(data) < cls.HEADER_SIZE + 1:
            raise ValueError(f"패킷 크기가 너무 작습니다: {len(data)} bytes")
        command_id = struct.unpack(cls.HEADER_FORMAT, data[:cls.HEADER_SIZE])[0].decode()
        payload = data[cls.HEADER_SIZE:-1]  # \n 제외
        end_byte = data[-1:]
        if end_byte != b'\n':
            raise ValueError("패킷의 종료 바이트가 유효하지 않습니다.")
        return cls(command_id, payload)

# === 통신 설정 ===

# 라즈베리파이와 Obstacle_server PC를 위한 서버 소켓 설정
RPI_SERVER_PORT = 3141
OBSTACLE_SERVER_PORT = 4040

# 전송할 소켓들을 저장할 변수
connections = {
    'RPI': None,
    'Obstacle': None
}

# 스레드 동기화를 위한 Lock
conn_lock = threading.Lock()

def handle_rpi_connection(conn, addr):
    global connections
    print(f"라즈베리파이 {addr} 에 연결되었습니다.")
    with conn_lock:
        connections['RPI'] = conn
    try:
        receive_video_and_process(conn, '라즈베리파이')
    except Exception as e:
        print(f"라즈베리파이 연결 처리 중 예외 발생: {e}")
    finally:
        with conn_lock:
            connections['RPI'] = None
        conn.close()
        print(f"라즈베리파이 {addr} 연결이 종료되었습니다.")

def handle_obstacle_connection(conn, addr):
    global connections
    print(f"Obstacle_server PC {addr} 에 연결되었습니다.")
    with conn_lock:
        connections['Obstacle'] = conn
    try:
        # Obstacle_server PC로 데이터를 전송하는 역할만 수행
        # 여기서는 수신을 하지 않으므로 블록킹 없이 유지
        while True:
            time.sleep(1)
            if conn.fileno() == -1:
                break
    except Exception as e:
        print(f"Obstacle_server PC 연결 처리 중 예외 발생: {e}")
    finally:
        with conn_lock:
            connections['Obstacle'] = None
        conn.close()
        print(f"Obstacle_server PC {addr} 연결이 종료되었습니다.")

# === 카메라 캘리브레이션 데이터 ===
# 카메라 매트릭스과 왜곡 계수 (예시 값)
mtx = np.array([[1.15753008e+03, 0.00000000e+00, 6.75382833e+02],
                [0.00000000e+00, 1.15189955e+03, 3.86729350e+02],
                [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
dist = np.array([[-0.26706898,  0.10305542, -0.00088013,  0.00080643, -0.19574027]])

# === 차선 검출 및 객체 감지 ===
def abs_sobel_thresh(img, orient='x', thresh_min=25, thresh_max=255):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32)

    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3))
    elif orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3))
    else:
        raise ValueError("orient must be 'x' or 'y'")

    if np.max(abs_sobel) == 0:
        scaled_sobel = np.uint8(abs_sobel)
    else:
        scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))

    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    return binary_output

def color_threshold(image, sthresh=(0, 255)):
    hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    l_channel = hls[:, :, 1]

    binary_output = np.zeros_like(l_channel)
    binary_output[(l_channel >= sthresh[0]) & (l_channel <= sthresh[1])] = 1

    return binary_output

def apply_roi(image):
    height, width = image.shape[:2]
    mask = np.zeros_like(image)

    polygon = np.array([[
        (0, int(height * 2/5)),
        (width, int(height * 2/5)),
        (width, height),
        (0, height)
    ]], np.int32)

    cv2.fillPoly(mask, polygon, 255)
    roi_image = cv2.bitwise_and(image, mask)
    return roi_image

# === 패킷 전송 함수 ===
def send_packet(sock, packet_bytes):
    try:
        sock.sendall(packet_bytes)
    except socket.error as e:
        print(f"패킷 전송 오류: {e}")

# === 라즈베리파이로 모터 명령 전송 함수 ===
def send_motor_commands_to_rpi(motor_value):
    with conn_lock:
        rpi_conn = connections['RPI']
    if rpi_conn:
        # 'MO' 명령 패킷 생성 (motor_value를 문자열로 변환하여 인코딩)
        motor_str = str(motor_value).zfill(4)  # 예: '1000'
        motor_data = motor_str.encode('utf-8')  # b'1000'
        packet = Packet('MO', motor_data)
        send_packet(rpi_conn, packet.to_bytes())
        print(f"라즈베리파이로 모터 명령 전송 완료: {motor_str}")
    else:
        print("라즈베리파이와의 연결이 존재하지 않습니다.")

# === Obstacle 서버로 영상 프레임 전송 함수 ===
def send_frame_to_obstacle(frame):
    with conn_lock:
        obs_conn = connections['Obstacle']
    if obs_conn:
        # 프레임 인코딩 (JPEG 형식)
        encoded_frame = cv2.imencode('.jpg', frame)[1].tobytes()
        
        # Obstacle 서버로 보낼 패킷 생성: 'SF' + frame_data + '\n'
        packet = Packet('SF', encoded_frame)
        packet_bytes = packet.to_bytes()
        
        send_packet(obs_conn, packet_bytes)
        print("Obstacle 서버로 영상 프레임 전송 완료.")
    else:
        print("Obstacle 서버와의 연결이 존재하지 않습니다.")

# === Obstacle 서버로 객체 검출 결과 전송 함수 ===
def send_detection_result_to_obstacle(result):
    with conn_lock:
        obs_conn = connections['Obstacle']
    if obs_conn:
        # 'DE' 명령 패킷 생성 (result를 문자열로 변환하여 인코딩)
        result_str = str(result)  # 예: '1' 또는 '0'
        detection_data = result_str.encode('utf-8')  # b'1'
        packet = Packet('DE', detection_data)
        send_packet(obs_conn, packet.to_bytes())
        print(f"Obstacle 서버로 객체 검출 결과 전송 완료: {result_str}")
    else:
        print("Obstacle 서버와의 연결이 존재하지 않습니다.")

# === 차선 인식 결과를 바탕으로 모터 값 결정 함수 ===
def determine_motor_value(preprocessImage):
    """
    차선 인식 결과를 바탕으로 모터 값을 결정
    예시 로직:
    - 차선이 중앙에 있으면 직진 (motor_value = 1000)
    - 차선이 왼쪽으로 치우치면 우회전 (motor_value = 1500)
    - 차선이 오른쪽으로 치우치면 좌회전 (motor_value = 500)
    """
    height, width = preprocessImage.shape
    midpoint = width // 2
    left_region = preprocessImage[int(height*0.6):, :midpoint]
    right_region = preprocessImage[int(height*0.6):, midpoint:]

    left_count = np.sum(left_region) / 255
    right_count = np.sum(right_region) / 255

    # 비교를 통해 차선의 위치를 판단
    if left_count > right_count + 50:
        motor_value = 1500  # 우회전
    elif right_count > left_count + 50:
        motor_value = 500   # 좌회전
    else:
        motor_value = 1000  # 직진

    print(f"차선 인식 결과에 따른 모터 값: {motor_value}")
    return motor_value

# === 라즈베리파이로부터 영상 패킷 수신 및 처리 ===
def receive_video_and_process(conn, device_name):
    buffer = b''
    try:
        while True:
            data = conn.recv(4096)
            if not data:
                print(f"{device_name} 연결이 종료되었습니다.")
                break
            buffer += data
            while b'\n' in buffer:
                packet_end = buffer.find(b'\n') + 1
                packet_data = buffer[:packet_end]
                buffer = buffer[packet_end:]
                try:
                    packet = Packet.from_bytes(packet_data)
                except ValueError as ve:
                    print(f"패킷 오류: {ve}")
                    continue

                if packet.command_id == 'SF':
                    # 프레임 데이터 추출 (앞 2바이트 명령 ID와 마지막 1바이트 \n 제외)
                    frame_data = packet.data  # 이미 앞 2바이트와 마지막 \n 제외됨

                    # 프레임 복원
                    frame = np.frombuffer(frame_data, dtype=np.uint8)
                    frame = cv2.imdecode(frame, cv2.IMREAD_COLOR)
                    if frame is None:
                        print("프레임 디코딩 실패")
                        continue

                    # === 카메라 캘리브레이션 적용 ===
                    undist_frame = cv2.undistort(frame, mtx, dist, None, mtx)

                    # === 차선 검출 ===
                    c_binary = color_threshold(undist_frame, sthresh=(200, 255))
                    gradx = abs_sobel_thresh(undist_frame, orient='x', thresh_min=20, thresh_max=100)
                    grady = abs_sobel_thresh(undist_frame, orient='y', thresh_min=20, thresh_max=100)

                    combined_binary = np.zeros_like(c_binary)
                    combined_binary[((gradx == 1) & (grady == 1)) & (c_binary == 1)] = 255

                    preprocessImage = apply_roi(combined_binary)

                    # === 차선 인식 결과 계산 ===
                    # 이미지 중간 지점 계산
                    midpoint = preprocessImage.shape[1] // 2

                    # 이진 이미지에서 차선 픽셀 좌표 추출
                    nonzero = preprocessImage.nonzero()
                    nonzeroy = np.array(nonzero[0])
                    nonzerox = np.array(nonzero[1])

                    # 왼쪽 및 오른쪽 차선 픽셀 식별
                    left_lane_inds = (nonzerox < midpoint)
                    right_lane_inds = (nonzerox >= midpoint)

                    # 왼쪽 차선 픽셀 좌표
                    leftx = nonzerox[left_lane_inds]
                    lefty = nonzeroy[left_lane_inds]

                    # 오른쪽 차선 픽셀 좌표
                    rightx = nonzerox[right_lane_inds]
                    righty = nonzeroy[right_lane_inds]

                    # 왼쪽 차선의 무게중심 계산 및 표시
                    if len(leftx) > 0 and len(lefty) > 0:
                        left_center_x = int(np.mean(leftx))
                        left_center_y = int(np.mean(lefty))
                        cv2.circle(undist_frame, (left_center_x, left_center_y), 5, (0, 255, 0), -1)  # 초록색 원
                    else:
                        left_center_x = None
                        left_center_y = None

                    # 오른쪽 차선의 무게중심 계산 및 표시
                    if len(rightx) > 0 and len(righty) > 0:
                        right_center_x = int(np.mean(rightx))
                        right_center_y = int(np.mean(righty))
                        cv2.circle(undist_frame, (right_center_x, right_center_y), 5, (0, 255, 0), -1)  # 초록색 원
                    else:
                        right_center_x = None
                        right_center_y = None

                    # 두 차선 무게중심의 중앙점 계산 및 표시
                    if left_center_x is not None and right_center_x is not None:
                        mid_center_x = int((left_center_x + right_center_x) / 2)
                        mid_center_y = int((left_center_y + right_center_y) / 2)
                        cv2.circle(undist_frame, (mid_center_x, mid_center_y), 5, (255, 0, 0), -1)  # 파란색 원
                    else:
                        mid_center_x = None
                        mid_center_y = None

                    # === 거리 계산 및 표시 ===
                    meters_per_pixel_top = 0.05  # 예시 값 (상단 픽셀당 미터)
                    meters_per_pixel_bottom = 0.05  # 예시 값 (하단 픽셀당 미터)

                    if left_center_x is not None and mid_center_x is not None and left_center_y is not None:
                        image_height = undist_frame.shape[0]
                        y_ratio = left_center_y / image_height

                        meters_per_pixel = meters_per_pixel_top + (meters_per_pixel_bottom - meters_per_pixel_top) * y_ratio

                        pixel_distance = abs(mid_center_x - left_center_x)

                        real_distance_meters = pixel_distance * meters_per_pixel

                        distance_text = f"Distance from Left Lane: {real_distance_meters:.2f} meters"

                        cv2.putText(undist_frame, distance_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                                    1, (255, 255, 255), 2, cv2.LINE_AA)
                    else:
                        cv2.putText(undist_frame, "Lane detection failed", (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                                    1, (0, 0, 255), 2, cv2.LINE_AA)

                    # === 객체 검출 로직 (예: 특정 색상 객체 감지) ===
                    hsv = cv2.cvtColor(undist_frame, cv2.COLOR_BGR2HSV)
                    # 예시: 빨간색 범위 (HSV)
                    lower_red1 = np.array([0, 70, 50])
                    upper_red1 = np.array([10, 255, 255])
                    lower_red2 = np.array([170, 70, 50])
                    upper_red2 = np.array([180, 255, 255])

                    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
                    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
                    mask = cv2.bitwise_or(mask1, mask2)

                    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                    object_detected = 0  # 기본값은 객체 미검출
                    if len(contours) > 0:
                        # 객체가 차선 내부에 있는지 확인
                        # 간단히 중앙 영역에 객체가 있는지 확인
                        central_region = preprocessImage[int(preprocessImage.shape[0]*0.6):, :]
                        for cnt in contours:
                            x, y, w, h = cv2.boundingRect(cnt)
                            # 좌표 변환: y는 이미지 상단부터 시작
                            if y > int(preprocessImage.shape[0]*0.6):
                                object_detected = 1
                                cv2.rectangle(undist_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                    # === 객체 검출 결과 전송 ===
                    send_detection_result_to_obstacle(object_detected)

                    # === 차선 인식 결과를 바탕으로 모터 값 전송 ===
                    motor_value = determine_motor_value(preprocessImage)
                    send_motor_commands_to_rpi(motor_value)

                    # === Obstacle 서버로 처리된 프레임 전송 ===
                    send_frame_to_obstacle(undist_frame)

                    # === 화면에 표시 (디버그용) ===
                    lane_overlay = np.zeros_like(undist_frame)
                    lane_overlay[preprocessImage == 255] = [0, 0, 255]
                    combined = cv2.addWeighted(undist_frame, 0.7, lane_overlay, 1.0, 0)

                    cv2.imshow('Lane Detection', combined)
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        print("사용자에 의해 종료됨.")
                        return
    except Exception as e:
        print(f"{device_name} 처리 중 예외 발생: {e}")
    finally:
        with conn_lock:
            if device_name == '라즈베리파이':
                connections['RPI'] = None
            elif device_name == 'Obstacle_server PC':
                connections['Obstacle'] = None
        conn.close()
        print(f"{device_name} 연결이 종료되었습니다.")

# === 서버 소켓 설정 함수 ===
def start_server(port, handler, device_name):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    try:
        server_socket.bind(('192.168.0.13', port))
        server_socket.listen(1)
        print(f"{device_name}의 연결을 기다리는 중... (포트 {port})\n")
    except socket.error as e:
        print(f"{device_name} 소켓 생성/바인딩 오류: {e}\n")
        server_socket.close()
        sys.exit(1)

    while True:
        try:
            conn, addr = server_socket.accept()
            handler_thread = threading.Thread(target=handler, args=(conn, addr))
            handler_thread.start()
        except Exception as e:
            print(f"{device_name} 소켓 수락 오류: {e}")
            server_socket.close()
            sys.exit(1)

# === 메인 실행 ===
if __name__ == "__main__":
    # 라즈베리파이 연결을 위한 서버 12345스레드 시작
    rpi_thread = threading.Thread(target=start_server, args=(RPI_SERVER_PORT, handle_rpi_connection, "라즈베리파이"))
    rpi_thread.start()

    # Obstacle_server PC 연결을 위한 서버 스레드 시작
    obstacle_thread = threading.Thread(target=start_server, args=(OBSTACLE_SERVER_PORT, handle_obstacle_connection, "Obstacle_server PC"))
    obstacle_thread.start()

    # 메인 스레드는 연결 대기를 계속
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("사용자에 의해 인터럽트됨.")
        # 소켓 종료는 각 핸들러에서 처리됨
        sys.exit(0)